---
## Python Packages & Directories
---

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind", 20)

# from smh import Session

from spag.read_data import *
from spag.convert import *
from spag.utils import *
from spag.calculate import *
import spag.read_data as rd
import spag.coordinates as scoord

# import alexmods.read_data as rd

script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"
data_dir = '/Users/ayelland/Research/metal-poor-stars/spag/spag/data/'

with open('/Users/ayelland/Research/metal-poor-stars/project/carbon-project-2025/create-tables-0-date.txt', 'r') as f:
    date = f.readline().strip()


In [30]:
## Show all columns and rows of the dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

---
## Load the CSV Datafiles
---

In [31]:
query_df = pd.read_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/hughes2025/obs_table.csv", comment="#")

---
## Preparing the Columns
---

In [32]:
target_stars = query_df.copy()
print(f"Total number of unique target stars: {len(target_stars)}")

Total number of unique target stars: 10


In [33]:
# ## Filter and Re-order the columns for the final target list
# auxcols = [
#     # 'Name', 
#     'Simbad_Identifier', 
#     'RA_hms', 
#     'DEC_dms', 
#     'RA_deg',
#     'DEC_deg',
#     'Loc', 
#     'System', 
#     'Teff',
#     'logg', 
#     'Vmic', 
#     'Fe/H', 
#     'M/H', 
#     'Class',
#     'f_ulc',
#     'f_ulfe',
#     'f_dtrans',
#     'Reference', 
# ]
# datacols = [
#     '[Fe/H]',
#     'ul[Fe/H]', 
#     '[C/H]',
#     'ul[C/H]', 
#     '[C/H]f', 
#     'ul[C/H]f', 
#     '[C/Fe]', 
#     'll[C/Fe]', 
#     'ul[C/Fe]',
#     '[C/Fe]f', 
#     'll[C/Fe]f', 
#     'ul[C/Fe]f', 
#     '[Ba/H]', 
#     'ul[Ba/H]', 
#     '[Ba/Fe]', 
#     'll[Ba/Fe]', 
#     'ul[Ba/Fe]', 
#     '[Sr/H]', 
#     'ul[Sr/H]', 
#     '[Sr/Fe]', 
#     'll[Sr/Fe]', 
#     'ul[Sr/Fe]',
#     '[Eu/H]', 
#     'ul[Eu/H]', 
#     '[Eu/Fe]', 
#     'll[Eu/Fe]',
#     'ul[Eu/Fe]', 
#     'Dtrans_l', 
#     'Dtrans_llim', 
#     'Dtrans_u', 
#     'Dtrans_ulim'
# ]
# target_stars = target_stars[auxcols + datacols].copy()


In [34]:
# ## Data columns
# data_cols = [col for col in list(target_stars.columns)[len(auxcols):] if col != 'epsc_c']

# ## Group columns intelligently
# grouped_cols = []
# i = 0
# while i < len(data_cols):
#     current_col = data_cols[i]
#     group = [current_col]
    
#     # Special handling for Dtrans columns
#     if current_col.startswith('Dtrans_'): # handling for Dtrans columns
        
#         if current_col.endswith('_l'):
#             # Look for corresponding _llim
#             base_name = current_col[:-2]  # Remove '_l'
#             llim_col = base_name + '_llim'
#             if i + 1 < len(data_cols) and data_cols[i + 1] == llim_col:
#                 group.append(data_cols[i + 1])
#                 i += 1
#         elif current_col.endswith('_u'):
#             # Look for corresponding _ulim  
#             base_name = current_col[:-2]  # Remove '_u'
#             ulim_col = base_name + '_ulim'
#             if i + 1 < len(data_cols) and data_cols[i + 1] == ulim_col:
#                 group.append(data_cols[i + 1])
#                 i += 1
#     else: # handling for ll/ul prefixed columns
    
#         # Check for lower limit column (ll prefix)
#         if i + 1 < len(data_cols) and data_cols[i + 1].startswith('ll'):
#             group.append(data_cols[i + 1])
#             i += 1
#         # Check for upper limit column (ul prefix)
#         if i + 1 < len(data_cols) and data_cols[i + 1].startswith('ul'):
#             group.append(data_cols[i + 1])
#             i += 1
    
#     grouped_cols.append(group)
#     i += 1

# display(grouped_cols)

# ## Merge the columns appropriately
# for group in grouped_cols:
#     base_col = group[0]
    
#     if len(group) == 2:  # Pair: base + ul
#         ul_col = group[1]
#         target_stars[base_col] = target_stars[base_col].fillna(
#             target_stars[ul_col].where(target_stars[ul_col].notna()).apply(lambda x: f'<{x}' if pd.notna(x) else np.nan)
#         )
#         target_stars.drop(columns=[ul_col], inplace=True)
        
#     elif len(group) == 3:  # Triplet: base + ll + ul
#         ll_col, ul_col = group[1], group[2]
#         target_stars[base_col] = target_stars[base_col].fillna(
#             target_stars[ll_col].where(target_stars[ll_col].notna()).apply(lambda x: f'>{x}' if pd.notna(x) else np.nan)
#         ).fillna(
#             target_stars[ul_col].where(target_stars[ul_col].notna()).apply(lambda x: f'<{x}' if pd.notna(x) else np.nan)
#         )
#         target_stars.drop(columns=[ll_col, ul_col], inplace=True)

# ## Merge the 'Fe/H' and 'M/H' columns into 'Fe/H'
# if 'M/H' in target_stars.columns and 'Fe/H' in target_stars.columns:
#     target_stars['M/H'] = target_stars['M/H'].fillna(target_stars['Fe/H'])
#     target_stars.drop(columns=['Fe/H'], inplace=True)

# # display(target_stars)

---
## Get the Magnitude Data by Querying Databases (Simbad, Vizier, 2MASS, Gaia, etc.)
---

In [35]:
query_by = 'identifier'
# query_by = 'coordinates'

identify_by = 'Name'
# identify_by = 'Simbad_Identifier'


### Query SIMBAD

In [36]:
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u

## Setup Simbad fields
Simbad.ROW_LIMIT = 10000
Simbad.add_votable_fields(
    'flux(U)', 'flux(B)', 'flux(V)', 'flux(R)', 'flux(I)', 'flux(J)', 'flux(H)', 'flux(K)',
    'otype', 'sp', 'ra', 'dec', 'pmra', 'pmdec', 'plx', 'rv_value'
)

## Iterate over target stars
results_list = []
for idx, row in target_stars.iterrows():

    if query_by == 'identifier':
        identifier = row.get(identify_by, f'coord_{idx}').strip()
        if 'RA_hms' in row and 'DEC_dms' in row:
            ra_hms = row['RA_hms']
            dec_dms = row['DEC_dms']
        try:
            result = Simbad.query_object(identifier)
            if result is not None:
                df = result.to_pandas()
                df['Found'] = True
                df['Query_ID'] = identifier
                df['RA_input'] = ra_hms if 'ra_hms' in locals() else None
                df['DEC_input'] = dec_dms if 'dec_dms' in locals() else None
            else:
                df = pd.DataFrame([{
                    'Found': False,
                    'Query_ID': identifier, 
                    'RA_input': ra_hms if 'ra_hms' in locals() else None,
                    'DEC_input': dec_dms if 'dec_dms' in locals() else None 
                }])
        except Exception as e:
            df = pd.DataFrame([{
                'Found': False, 
                'Query_ID': identifier,
                'RA_input': ra_hms if 'ra_hms' in locals() else None,
                'DEC_input': dec_dms if 'dec_dms' in locals() else None,
                'Error': str(e)
            }])
        results_list.append(df)

    elif query_by == 'coordinates':
        identifier = row.get(identify_by, f'coord_{idx}').strip()
        ra_hms = row['RA_hms']
        dec_dms = row['DEC_dms']
        try:
            coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=(u.hourangle, u.deg))
            result = Simbad.query_region(coord, radius='5s')
            if result is not None:
                df = result.to_pandas()
                df['Found'] = True
                df['Query_ID'] = identifier
                df['RA_input'] = ra_hms
                df['DEC_input'] = dec_dms
            else:
                df = pd.DataFrame([{
                    'Found': False,
                    'Query_ID': identifier,
                    'RA_input': ra_hms,
                    'DEC_input': dec_dms
                }])
        except Exception as e:
            df = pd.DataFrame([{
                'Found': False,
                'Query_ID': identifier,
                'RA_input': ra_hms,
                'DEC_input': dec_dms,
                'Error': str(e)
            }])
        results_list.append(df)

# Combine results and reorder columns
simbad_df = pd.concat(results_list, ignore_index=True)

priority_cols = ['Found', 'Query_ID', 'RA_input', 'DEC_input']
for col in priority_cols:
    if col not in simbad_df.columns:
        simbad_df[col] = pd.NA
cols = simbad_df.columns.tolist()

for col in reversed(priority_cols):
    if col in cols:
        cols.insert(0, cols.pop(cols.index(col)))
simbad_df = simbad_df[cols]

In [37]:
display(simbad_df)
display(list(simbad_df.columns))

,Found,Query_ID,RA_input,DEC_input,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,FLUX_U,FLUX_B,FLUX_V,FLUX_R,FLUX_I,FLUX_J,FLUX_H,FLUX_K,OTYPE,SP_TYPE,RA_2,DEC_2,PMRA,PMDEC,PLX_VALUE,RV_VALUE,FLUX_U_1,FLUX_B_1,FLUX_V_1,FLUX_R_1,FLUX_I_1,FLUX_J_1,FLUX_H_1,FLUX_K_1,OTYPE_2,SP_TYPE_2,RA_3,DEC_3,PMRA_2,PMDEC_2,PLX_VALUE_2,RV_VALUE_2,FLUX_U_2,FLUX_B_2,FLUX_V_2,FLUX_R_2,FLUX_I_2,FLUX_J_2,FLUX_H_2,FLUX_K_2,OTYPE_3,SP_TYPE_3,RA_4,DEC_4,PMRA_3,PMDEC_3,PLX_VALUE_3,RV_VALUE_3,SCRIPT_NUMBER_ID
0,True,2MASS J01234438-7332025,NaN,NaN,2MASS J01234438-7332025,01 23 44.3701,-73 32 02.524,14.0,14.0,0.0098,0.0082,90.0,A,O,2020yCat.1350....0G,NaN,13.650,12.550,NaN,NaN,10.492,9.880,9.767,ChemPec*,,01 23 44.3701,-73 32 02.524,5.142,-0.812,0.0995,-57.00000,NaN,13.650,12.550,NaN,NaN,10.492,9.880,9.767,ChemPec*,,01 23 44.3701,-73 32 02.524,5.142,-0.812,0.0995,-57.00000,NaN,13.650,12.550,NaN,NaN,10.492,9.880,9.767,ChemPec*,,01 23 44.3701,-73 32 02.524,5.142,-0.812,0.0995,-57.00000,1.0
1,True,2MASS J14362653-3127262,NaN,NaN,UCAC4 293-078318,14 36 26.5359,-31 27 26.222,14.0,14.0,0.0155,0.0151,90.0,A,O,2020yCat.1350....0G,NaN,14.725,13.635,13.483,NaN,11.435,10.820,10.688,ChemPec*,,14 36 26.5359,-31 27 26.222,-2.750,-1.208,0.0051,199.90516,NaN,14.725,13.635,13.483,NaN,11.435,10.820,10.688,ChemPec*,,14 36 26.5359,-31 27 26.222,-2.750,-1.208,0.0051,199.90516,NaN,14.725,13.635,13.483,NaN,11.435,10.820,10.688,ChemPec*,,14 36 26.5359,-31 27 26.222,-2.750,-1.208,0.0051,199.90516,1.0
2,True,2MASS J20055457-0928302,NaN,NaN,ATO J301.4773-09.4750,20 05 54.5796,-09 28 30.157,14.0,14.0,0.0221,0.0167,90.0,A,O,2020yCat.1350....0G,NaN,15.184,13.492,12.966,NaN,8.998,8.077,7.723,LongPeriodV*,,20 05 54.5796,-09 28 30.157,-7.119,-6.241,0.0869,-51.74000,NaN,15.184,13.492,12.966,NaN,8.998,8.077,7.723,LongPeriodV*,,20 05 54.5796,-09 28 30.157,-7.119,-6.241,0.0869,-51.74000,NaN,15.184,13.492,12.966,NaN,8.998,8.077,7.723,LongPeriodV*,,20 05 54.5796,-09 28 30.157,-7.119,-6.241,0.0869,-51.74000,1.0
3,False,Gaia DR3 4348475068024891776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,Gaia DR3 6308927046493238528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,False,Gaia DR3 6725661847747358336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,True,Gaia DR3 6726207068062588160,NaN,NaN,Gaia DR3 6726207068062588160,18 21 50.8206,-41 03 33.574,14.0,14.0,0.0118,0.0106,90.0,A,O,2020yCat.1350....0G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChemPec*,,18 21 50.8206,-41 03 33.574,-6.974,-12.936,0.1564,24.26000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChemPec*,,18 21 50.8206,-41 03 33.574,-6.974,-12.936,0.1564,24.26000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChemPec*,,18 21 50.8206,-41 03 33.574,-6.974,-12.936,0.1564,24.26000,1.0
7,True,SMSS J030634.26-750133.3,NaN,NaN,SMSS J030634.26-750133.3,03 06 34.2706,-75 01 33.199,14.0,14.0,0.0119,0.0110,90.0,A,O,2020yCat.1350....0G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChemPec*,,03 06 34.2706,-75 01 33.199,9.259,-7.486,0.1879,145.83000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChemPec*,,03 06 34.2706,-75 01 33.199,9.259,-7.486,0.1879,145.83000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChemPec*,,03 06 34.2706,-75 01 33.199,9.259,-7.486,0.1879,145.83000,1.0
8,True,SMSS J030740.92-610018.8,NaN,NaN,SMSS J030740.92-610018.8,03 07 40.9499,-61 00 18.764,14.0,14.0,0.0148,0.0130,90.0,A,O,2020yCat.1350....0G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChemPec*,,03 07 

['Found',
 'Query_ID',
 'RA_input',
 'DEC_input',
 'MAIN_ID',
 'RA',
 'DEC',
 'RA_PREC',
 'DEC_PREC',
 'COO_ERR_MAJA',
 'COO_ERR_MINA',
 'COO_ERR_ANGLE',
 'COO_QUAL',
 'COO_WAVELENGTH',
 'COO_BIBCODE',
 'FLUX_U',
 'FLUX_B',
 'FLUX_V',
 'FLUX_R',
 'FLUX_I',
 'FLUX_J',
 'FLUX_H',
 'FLUX_K',
 'OTYPE',
 'SP_TYPE',
 'RA_2',
 'DEC_2',
 'PMRA',
 'PMDEC',
 'PLX_VALUE',
 'RV_VALUE',
 'FLUX_U_1',
 'FLUX_B_1',
 'FLUX_V_1',
 'FLUX_R_1',
 'FLUX_I_1',
 'FLUX_J_1',
 'FLUX_H_1',
 'FLUX_K_1',
 'OTYPE_2',
 'SP_TYPE_2',
 'RA_3',
 'DEC_3',
 'PMRA_2',
 'PMDEC_2',
 'PLX_VALUE_2',
 'RV_VALUE_2',
 'FLUX_U_2',
 'FLUX_B_2',
 'FLUX_V_2',
 'FLUX_R_2',
 'FLUX_I_2',
 'FLUX_J_2',
 'FLUX_H_2',
 'FLUX_K_2',
 'OTYPE_3',
 'SP_TYPE_3',
 'RA_4',
 'DEC_4',
 'PMRA_3',
 'PMDEC_3',
 'PLX_VALUE_3',
 'RV_VALUE_3',
 'SCRIPT_NUMBER_ID']

### Query Gaia DR3

In [38]:
# example: query Gaia DR3 by Gaia source_id string using astroquery.gaia
from astroquery.gaia import Gaia
import pandas as pd
import re

# optional: set a larger timeout if needed
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"   # not required but documents intent
Gaia.ROW_LIMIT = -1

def query_gaia_by_identifier(identifier, columns=None):
    """
    identifier: str, e.g. "Gaia DR3 5480105831331104384" or just the numeric id as str/int
    columns: list or None, ADQL column names to request (None -> request a small useful set)
    returns: pandas.DataFrame (possibly empty) or raises on severe errors
    """
    # try to extract numeric source_id from common forms
    m = re.search(r'(\d{9,22})', str(identifier))  # Gaia DR3 source_id is a long int (>=9 digits)
    if not m:
        raise ValueError(f"Could not find a numeric Gaia source_id in '{identifier}'")

    source_id = int(m.group(1))
    # default columns if none specified
    if columns is None:
        columns = [
            "source_id", 
            "ra", 
            "dec", 
            "parallax", 
            "parallax_error",
            "pmra",
            "pmdec", 
            "radial_velocity",
            "phot_g_mean_mag", 
            "phot_bp_mean_mag", 
            "phot_rp_mean_mag"
        ]
    col_str = ", ".join(columns)

    adql = f"SELECT {col_str} FROM gaiadr3.gaia_source WHERE source_id = {source_id}"

    job = Gaia.launch_job_async(adql)      # launches ADQL job
    tbl = job.get_results()
    if len(tbl) == 0:
        return pd.DataFrame()             # empty -> not found
    df = tbl.to_pandas()
    return df

def query_gaia_by_coordinates(ra_deg, dec_deg, radius_arcsec=2, columns=None):
    """
    ra_deg, dec_deg: float, coordinates in degrees
    radius_arcsec: float, search radius in arcseconds
    columns: list or None, ADQL column names to request (None -> request a small useful set)
    returns: pandas.DataFrame (possibly empty) or raises on severe errors
    """
    if columns is None:
        columns = [
            "source_id", 
            "ra", 
            "dec", 
            "parallax", 
            "parallax_error",
            "pmra",
            "pmdec", 
            "radial_velocity",
            "phot_g_mean_mag", 
            "phot_bp_mean_mag", 
            "phot_rp_mean_mag"
        ]
    col_str = ", ".join(columns)

    radius_deg = radius_arcsec / 3600.0
    adql = f"""
    SELECT {col_str} 
    FROM gaiadr3.gaia_source 
    WHERE 1=CONTAINS(
        POINT('ICRS', ra, dec), 
        CIRCLE('ICRS', {ra_deg}, {dec_deg}, {radius_deg})
    )
    """

    job = Gaia.launch_job_async(adql)      # launches ADQL job
    tbl = job.get_results()
    if len(tbl) == 0:
        return pd.DataFrame()             # empty -> not found
    df = tbl.to_pandas()
    return df

## Iterate over target stars to query Gaia by identifier or coordinates
results_list = []
for idx, row in target_stars.iterrows():
    if query_by == 'identifier':
        identifier = row.get(identify_by, f'coord_{idx}').strip()
        try:
            result = query_gaia_by_identifier(identifier)
            if result.empty:
                df = pd.DataFrame([{
                    'Found': False,
                    'Query_ID': identifier
                }])
            else:
                df = result.copy()
                df['Found'] = True
                df['Query_ID'] = identifier
        except Exception as e:
            df = pd.DataFrame([{
                'Found': False,
                'Query_ID': identifier,
                'Error': str(e)
            }])
        results_list.append(df)

    elif query_by == 'coordinates':
        identifier = row.get(identify_by, f'coord_{idx}').strip()
        ra_deg = row['RA_deg']
        dec_deg = row['DEC_deg']
        try:
            result = query_gaia_by_coordinates(ra_deg, dec_deg, radius_arcsec=2)
            if result.empty:
                df = pd.DataFrame([{
                    'Found': False,
                    'Query_ID': identifier,
                    'RA_input': ra_deg,
                    'DEC_input': dec_deg
                }])
            else:
                df = result.copy()
                df['Found'] = True
                df['Query_ID'] = identifier
                df['RA_input'] = ra_deg
                df['DEC_input'] = dec_deg
        except Exception as e:
            df = pd.DataFrame([{
                'Found': False,
                'Query_ID': identifier,
                'RA_input': ra_deg,
                'DEC_input': dec_deg,
                'Error': str(e)
            }])
        results_list.append(df)

# Combine results and reorder columns
gaia_df = pd.concat(results_list, ignore_index=True)

priority_cols = ['Found', 'Query_ID', 'RA_input', 'DEC_input']
for col in priority_cols:
    if col not in gaia_df.columns:
        gaia_df[col] = pd.NA
cols = gaia_df.columns.tolist()

for col in reversed(priority_cols):
    if col in cols:
        cols.insert(0, cols.pop(cols.index(col)))
gaia_df = gaia_df[cols]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [39]:
display(gaia_df)

,Found,Query_ID,RA_input,DEC_input,Error,SOURCE_ID,ra,dec,parallax,parallax_error,pmra,pmdec,radial_velocity,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag
0,False,2MASS J01234438-7332025,<NA>,<NA>,Could not find a numeric Gaia source_id in '2M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,2MASS J14362653-3127262,<NA>,<NA>,Could not find a numeric Gaia source_id in '2M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,2MASS J20055457-0928302,<NA>,<NA>,Could not find a numeric Gaia source_id in '2M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,Gaia DR3 4348475068024891776,<NA>,<NA>,NaN,4.348475e+18,242.634589,-8.260693,0.158258,0.015675,10.612852,-5.897113,-142.247635,13.229983,13.778419,12.513052
4,True,Gaia DR3 6308927046493238528,<NA>,<NA>,NaN,6.308927e+18,226.258881,-13.738954,0.047575,0.022725,-0.250828,-7.449533,-237.681168,13.733918,14.267475,13.036109
5,True,Gaia DR3 6725661847747358336,<NA>,<NA>,NaN,6.725662e+18,272.431692,-40.868886,0.170048,0.016993,-10.090384,-14.046178,466.002136,13.278628,13.768646,12.611086
6,True,Gaia DR3 6726207068062588160,<NA>,<NA>,NaN,6.726207e+18,275.461712,-41.059384,0.156433,0.013717,-6.974008,-12.935874,24.261440,13.227261,13.672575,12.598391
7,False,SMSS J030634.26-750133.3,<NA>,<NA>,Could not find a numeric Gaia source_id in 'SM...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,False,SMSS J030740.92-610018.8,<NA>,<NA>,Could not find a numeric Gaia source_id in 'SM...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,False,SMSS J090921.70-163821.1,<NA>,<NA>,Could not find a numeric Gaia source_id in 'SM...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
display(gaia_df.columns)

Index(['Found', 'Query_ID', 'RA_input', 'DEC_input', 'Error', 'SOURCE_ID',
       'ra', 'dec', 'parallax', 'parallax_error', 'pmra', 'pmdec',
       'radial_velocity', 'phot_g_mean_mag', 'phot_bp_mean_mag',
       'phot_rp_mean_mag'],
      dtype='object')

### Merge the Magnitude Data into the Target List

In [72]:
target_stars_out_cols = list(set(simbad_df.columns.tolist() + gaia_df.columns.tolist()))
target_stars_out = pd.DataFrame(columns=target_stars_out_cols)

for i, row in target_stars.iterrows():
    identifier = row.get(identify_by, f'coord_{i}').strip()
    simbad_matches = simbad_df[simbad_df['Query_ID'] == identifier]
    gaia_matches = gaia_df[gaia_df['Query_ID'] == identifier]

    print(i, identifier)
    target_stars_out.at[i, 'Query_ID'] = identifier
    if not simbad_matches.empty:
        for col in simbad_matches.columns:
            # if col in ['FLUX_U','FLUX_B','FLUX_V','FLUX_R','FLUX_I','FLUX_J','FLUX_H','FLUX_K']:
            #     target_stars_out.at[i, col.replace('FLUX_', '')+'mag'] = normal_round(simbad_matches.iloc[0][col], 1)
            target_stars_out.at[i, col] = simbad_matches.iloc[0][col]

    if not gaia_matches.empty:
        for col in gaia_matches.columns:
            # if col in ['phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag']:
            #     target_stars_out.at[i, col.replace('phot_', '').replace('_mean_mag', '').upper()+'mag'] = normal_round(gaia_matches.iloc[0][col], 1)
            target_stars_out.at[i, col] = gaia_matches.iloc[0][col]
            
display(sorted(list(target_stars_out.columns)))
# display(target_stars_out)

0 2MASS J01234438-7332025
1 2MASS J14362653-3127262
2 2MASS J20055457-0928302
3 Gaia DR3 4348475068024891776
4 Gaia DR3 6308927046493238528
5 Gaia DR3 6725661847747358336
6 Gaia DR3 6726207068062588160
7 SMSS J030634.26-750133.3
8 SMSS J030740.92-610018.8
9 SMSS J090921.70-163821.1


['COO_BIBCODE',
 'COO_ERR_ANGLE',
 'COO_ERR_MAJA',
 'COO_ERR_MINA',
 'COO_QUAL',
 'COO_WAVELENGTH',
 'DEC',
 'DEC_2',
 'DEC_3',
 'DEC_4',
 'DEC_PREC',
 'DEC_input',
 'Error',
 'FLUX_B',
 'FLUX_B_1',
 'FLUX_B_2',
 'FLUX_H',
 'FLUX_H_1',
 'FLUX_H_2',
 'FLUX_I',
 'FLUX_I_1',
 'FLUX_I_2',
 'FLUX_J',
 'FLUX_J_1',
 'FLUX_J_2',
 'FLUX_K',
 'FLUX_K_1',
 'FLUX_K_2',
 'FLUX_R',
 'FLUX_R_1',
 'FLUX_R_2',
 'FLUX_U',
 'FLUX_U_1',
 'FLUX_U_2',
 'FLUX_V',
 'FLUX_V_1',
 'FLUX_V_2',
 'Found',
 'MAIN_ID',
 'OTYPE',
 'OTYPE_2',
 'OTYPE_3',
 'PLX_VALUE',
 'PLX_VALUE_2',
 'PLX_VALUE_3',
 'PMDEC',
 'PMDEC_2',
 'PMDEC_3',
 'PMRA',
 'PMRA_2',
 'PMRA_3',
 'Query_ID',
 'RA',
 'RA_2',
 'RA_3',
 'RA_4',
 'RA_PREC',
 'RA_input',
 'RV_VALUE',
 'RV_VALUE_2',
 'RV_VALUE_3',
 'SCRIPT_NUMBER_ID',
 'SOURCE_ID',
 'SP_TYPE',
 'SP_TYPE_2',
 'SP_TYPE_3',
 'dec',
 'parallax',
 'parallax_error',
 'phot_bp_mean_mag',
 'phot_g_mean_mag',
 'phot_rp_mean_mag',
 'pmdec',
 'pmra',
 'ra',
 'radial_velocity']

In [74]:
## Sort the Columns
priority_cols = ['Query_ID', 'Found', 'RA_input', 'DEC_input']
coord_cols = [col for col in target_stars_out.columns if any(sub in col for sub in ['RA', 'DEC', 'ra', 'dec'])]
other_cols = [col for col in target_stars_out.columns if col not in priority_cols + coord_cols]
sorted_cols = priority_cols + coord_cols + other_cols
target_stars_out = target_stars_out[sorted_cols]

display(target_stars_out)

,Query_ID,Found,RA_input,DEC_input,PMRA_2,DEC_input,RA_4,pmra,PMRA,RA_input,RA_3,RA,RA_PREC,parallax_error,DEC_4,RA_2,DEC_2,DEC_PREC,DEC,DEC_3,ra,PMDEC,PMDEC_2,PMRA_3,parallax,pmdec,PMDEC_3,radial_velocity,dec,Error,FLUX_B_1,FLUX_U,FLUX_B_2,FLUX_K_2,phot_bp_mean_mag,FLUX_H_1,COO_BIBCODE,phot_rp_mean_mag,FLUX_J,SP_TYPE_2,FLUX_J_1,phot_g_mean_mag,COO_WAVELENGTH,SOURCE_ID,FLUX_K_1,FLUX_H_2,FLUX_J_2,FLUX_H,PLX_VALUE,OTYPE_2,FLUX_K,PLX_VALUE_2,FLUX_V_2,OTYPE_3,FLUX_V_1,COO_ERR_MINA,FLUX_I_2,FLUX_R,FLUX_R_2,COO_ERR_ANGLE,FLUX_U_1,SP_TYPE,COO_ERR_MAJA,FLUX_U_2,COO_QUAL,RV_VALUE_3,SP_TYPE_3,FLUX_B,FLUX_R_1,MAIN_ID,PLX_VALUE_3,SCRIPT_NUMBER_ID,FLUX_I_1,FLUX_V,FLUX_I,RV_VALUE,RV_VALUE_2,OTYPE
0,2MASS J01234438-7332025,False,<NA>,<NA>,5.142,<NA>,01 23 44.3701,NaN,5.142,<NA>,01 23 44.3701,01 23 44.3701,14.0,NaN,-73 32 02.524,01 23 44.3701,-73 32 02.524,14.0,-73 32 02.524,-73 32 02.524,NaN,-0.812,-0.812,5.142,NaN,NaN,-0.812,NaN,NaN,Could not find a numeric Gaia source_id in '2M...,13.65,NaN,13.65,9.767,NaN,9.88,2020yCat.1350....0G,NaN,10.492,,10.492,NaN,O,NaN,9.767,9.88,10.492,9.88,0.0995,ChemPec*,9.767,0.0995,12.55,ChemPec*,12.55,0.0082,NaN,NaN,NaN,90.0,NaN,,0.0098,NaN,A,-57.0,,13.65,NaN,2MASS J01234438-7332025,0.0995,1.0,NaN,12.55,NaN,-57.0,-57.0,ChemPec*
1,2MASS J14362653-3127262,False,<NA>,<NA>,-2.75,<NA>,14 36 26.5359,NaN,-2.75,<NA>,14 36 26.5359,14 36 26.5359,14.0,NaN,-31 27 26.222,14 36 26.5359,-31 27 26.222,14.0,-31 27 26.222,-31 27 26.222,NaN,-1.208,-1.208,-2.75,NaN,NaN,-1.208,NaN,NaN,Could not find a numeric Gaia source_id in '2M...,14.725,NaN,14.725,10.688,NaN,10.82,2020yCat.1350....0G,NaN,11.435,,11.435,NaN,O,NaN,10.688,10.82,11.435,10.82,0.0051,ChemPec*,10.688,0.0051,13.635,ChemPec*,13.635,0.0151,NaN,13.483,13.483,90.0,NaN,,0.0155,NaN,A,199.90516,,14.725,13.483,UCAC4 293-078318,0.0051,1.0,NaN,13.635,NaN,199.90516,199.90516,ChemPec*
2,2MASS J20055457-0928302,False,<NA>,<NA>,-7.119,<NA>,20 05 54.5796,NaN,-7.119,<NA>,20 05 54.5796,20 05 54.5796,14.0,NaN,-09 28 30.157,20 05 54.5796,-09 28 30.157,14.0,-09 28 30.157,-09 28 30.157,NaN,-6.241,-6.241,-7.119,NaN,NaN,-6.241,NaN,NaN,Could not find a numeric Gaia source_id in '2M...,15.184,NaN,15.184,7.723,NaN,8.077,2020yCat.1350....0G,NaN,8.998,,8.998,NaN,O,NaN,7.723,8.077,8.998,8.077,0.0869,LongPeriodV*,7.723,0.0869,13.492,LongPeriodV*,13.492,0.0167,NaN,12.966,12.966,90.0,NaN,,0.0221,NaN,A,-51.74,,15.184,12.966,ATO J301.4773-09.4750,0.0869,1.0,NaN,13.492,NaN,-51.74,-51.74,LongPeriodV*
3,Gaia DR3 4348475068024891776,True,<NA>,<NA>,NaN,<NA>,NaN,10.612852,NaN,<NA>,NaN,NaN,NaN,0.015675,NaN,NaN,NaN,NaN,NaN,NaN,242.634589,NaN,NaN,NaN,0.158258,-5.897113,NaN,-142.247635,-8.260693,NaN,NaN,NaN,NaN,NaN,13.778419,NaN,NaN,12.513052,NaN,NaN,NaN,13.229983,NaN,4348475068024891904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gaia DR3 6308927046493238528,True,<NA>,<NA>,NaN,<NA>,NaN,-0.250828,NaN,<NA>,NaN,NaN,NaN,0.022725,NaN,NaN,NaN,NaN,NaN,NaN,226.258881,NaN,NaN,NaN,0.047575,-7.449533,NaN,-237.681168,-13.738954,NaN,NaN,NaN,NaN,NaN,14.267475,NaN,NaN,13.036109,NaN,NaN,NaN,13.733918,NaN,6308927046493238272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Gaia DR3 6725661847747358336,True,<NA>,<NA>,NaN,<NA>,NaN,-10.090384,NaN,<NA>,NaN,NaN,NaN,0.016993,NaN,NaN,NaN,NaN,NaN,NaN,272.431692,NaN,NaN,NaN,0.170048,-14.046178,NaN,466.002136,-40.868886,NaN,NaN,NaN,NaN,NaN,13.768646,NaN,NaN,12.611086,NaN,NaN,NaN,13.278628,NaN,6725661847747358720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Gaia DR3 6726207068062588160,True,<NA>,<NA>,-6.974,<NA>,18 21 50.8206,-6.974008,-6.974,<NA>,18 21 50.8206,18 21 50.8206,14.0,0.013717,-41 03 33.574,18 21 50.8206,-41 03 33.574,14.0,-41 03 33.574,-41 03 33.574,275.461712,-12.936,-12.936,-6.974,0.156433,-12.93

---
## Save the Final Target List as a CSV & FWF
---

In [75]:
## Sort by RA & Dec
# target_stars_out = target_stars_out.sort_values(by=['RA_hms', 'DEC_dms']).reset_index(drop=True)

## Save the target list to a CSV file
target_stars_out.to_csv(os.path.join(script_dir, 'target_stars.csv'), index=False)

## Save the target list to a fixed-width file
target_stars_out = target_stars_out.fillna("---")
target_stars_out.to_string(buf=os.path.join(script_dir, 'target_stars.txt'), index=False)
